## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Jamestown,42.10,-79.24,78.01,53,1,8.05,US,clear sky
1,1,La Ronge,55.10,-105.28,66.20,72,75,16.11,CA,broken clouds
2,2,Belaya Gora,68.53,146.42,40.46,80,99,5.55,RU,overcast clouds
3,3,Hovd,48.01,91.64,59.31,38,29,8.97,MN,scattered clouds
4,4,Tura,25.52,90.22,72.86,94,86,4.03,IN,overcast clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Please enter the minimum temperature params: '))
max_temp= float(input('Please enter the maximum temperature params'))

Please enter the minimum temperature params: 70
Please enter the maximum temperature params90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_df = city_data_df.loc[(city_data_df['Max Temp']<=max_temp)&( city_data_df['Max Temp']>=70)]
filtered_city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Jamestown,42.10,-79.24,78.01,53,1,8.05,US,clear sky
4,4,Tura,25.52,90.22,72.86,94,86,4.03,IN,overcast clouds
5,5,Vaini,-21.20,-175.20,78.80,83,20,11.41,TO,few clouds
9,9,Sakakah,29.97,40.21,84.20,14,0,6.93,SA,clear sky
11,11,Sampit,-2.53,112.95,74.55,96,84,2.24,ID,broken clouds


In [17]:
# 4a. Determine if there are any empty rows.
filtered_city_df.count()

Unnamed: 0     319
City           319
Lat            319
Lng            319
Max Temp       319
Humidity       319
Cloudiness     319
Wind Speed     319
Country        317
Description    319
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_df.dropna(inplace=True)

<ipython-input-18-da90df48591d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_city_df.dropna(inplace=True)


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_city_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Jamestown,US,78.01,clear sky,42.10,-79.24,
4,Tura,IN,72.86,overcast clouds,25.52,90.22,
5,Vaini,TO,78.80,few clouds,-21.20,-175.20,
9,Sakakah,SA,84.20,clear sky,29.97,40.21,
11,Sampit,ID,74.55,broken clouds,-2.53,112.95,
14,Matara,LK,79.45,light rain,5.95,80.54,
19,Albany,US,84.00,broken clouds,42.60,-73.97,
21,Paragominas,BR,85.50,broken clouds,-2.97,-47.48,
22,Faanui,PF,77.14,light rain,-16.48,-151.75,
23,Touros,BR,75.20,few clouds,-5.20,-35.46,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # 6c. Get latitude and longitude from DataFrame
    params['location']= f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels= requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"]= hotels['results'][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name']!= ''].copy()
clean_hotel_df.count()

City           303
Country        303
Max Temp       303
Description    303
Lat            303
Lng            303
Hotel Name     303
dtype: int64

In [37]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30,31),zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))